In [1]:
from andbro__querrySeismoData import __querrySeismoData
from obspy import UTCDateTime, Stream

import matplotlib.pyplot as plt

In [2]:
stationlist = ['GELB','GRMB','BIB','TON', 'ALFT', 'FFB1', 'FFB2', 'FFB3', 'FUR']

tbeg = UTCDateTime("2021-03-04 18:30")
tend = UTCDateTime("2021-03-04 19:30")


In [ ]:
st = Stream()

for station in stationlist: 
    
    print(f"\nloading {station} ...")
    
    if station == "FUR":
        seed_id = f"GR.{station}..BHZ"
    else:
        seed_id = f"BW.{station}..BHZ"
    
    try:
        st0, inv = __querrySeismoData(
                                    seed_id=seed_id,
                                    starttime=tbeg-1,
                                    endtime=tend+2,
                                    where="online",
                                    path=None,
                                    restitute=False,
                                    detail=None,
                                    )
        

    except:
        print(f"-> failed obtaining data for station: {station}")
    
    if 'st0' in globals():
        try:
            st0.remove_response(
                        inventory=inv, 
                        pre_filt=[0.001, 0.004, 45, 50],
                        output="DISP",
                        )
        except:
            print(f"-> failed removing response for station: {station}")

        
        ## merging to one stream
        st += st0
        del st0



loading GELB ...


In [ ]:
st.trim(tbeg, tend) ## try to equalize amount of samples

for tr in st:
    tr.resample(20)
    
st

In [ ]:

def __get_distance(st1, st2, unit="m"):

    from obspy.geodetics.base import gps2dist_azimuth
    
    if str(type(st[0])) == "<class 'obspy.core.trace.Trace'>":
        lat1, lon1 = st1.stats.coordinates.latitude, st1.stats.coordinates.longitude
        lat2, lon2 = st2.stats.coordinates.latitude, st2.stats.coordinates.longitude
    elif str(type(st[0])) == "<class 'obspy.core.stream.Stream'>":
        lat1, lon1 = st1[0].stats.coordinates.latitude, st1[0].stats.coordinates.longitude
        lat2, lon2 = st2[0].stats.coordinates.latitude, st2[0].stats.coordinates.longitude        
    
    dist, az, baz = gps2dist_azimuth(lat1, lon1, lat2, lon2) # WGS84
    
    if unit == "m":
        return dist
    elif unit == "km":
        return (dist / 1000) 

dist = __get_distance(st[0], st[1], unit="km")




In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15,8))


ax[0].plot(st[0].data)

ax[1].plot(st[0].data)

ax[2].plot((st[0].data - st[1].data) / __get_distance(st[0], st[1]))